<a href="https://colab.research.google.com/github/eladses/sudoku-solver/blob/main/sudoku_solver_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

**in development!! not done**

sudoku solver using openCV, pytesseract and py-sudoku to find solution of sudoku from image

---

**How To Use**

1. upload image
2. run imports
3. run algorethms

---

**How It Works**

1. extraction board
  1. finding biggest square
  2. bla bla
2. filtering cells
  1. bla bla
  2. more bla


# Upload

In [ ]:
sudoku_url = 'https://www.google.com/search?q=sudoku+from+newspaper&sca_esv=abf976e12d8dcb8e&sca_upv=1&udm=2&biw=1920&bih=911&sxsrf=ADLYWIKsYXvSNuKkCf2_JK5OwxPZg1t-lg%3A1715267377075&ei=Mec8Zu6gBOz1i-gPiPCJuAY&oq=sudoku+from+&gs_lp=Egxnd3Mtd2l6LXNlcnAiDHN1ZG9rdSBmcm9tICoCCAAyBRAAGIAEMgcQABiABBgYMgcQABiABBgYSOcvUJ0UWNAkcAF4AJABAJgBggGgAZoLqgEEMC4xMrgBA8gBAPgBAZgCDaACvwvCAgQQIxgnwgIKEAAYgAQYQxiKBcICDhAAGIAEGLEDGIMBGIoFwgIIEAAYgAQYsQOYAwCIBgGSBwQxLjEyoAe-Ow&sclient=gws-wiz-serp#vhid=1C_w0RtkfCNCuM&vssid=mosaic'

# Imports

In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install py-sudoku
!pip install opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,455 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [3]:
from sudoku import Sudoku

import pytesseract
# pytesseract.pytesseract.tesseract_cmd = (
#     r'/usr/bin/tesseract'
# )

from PIL import Image

import numpy as np

from imutils import contours
import cv2
from google.colab.patches import cv2_imshow

from tensorflow import keras
import matplotlib.pyplot as plt


# from PIL import Image

# computer vision functions

In [ ]:
def get_board(img):
    """corpes the board from image

    Args:
      img (cv2 image format) - the image of the sudocu

    Returns:
      board_img (cv2 image format) - the board corped for the image
    """
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    ret,thresh = cv2.threshold(gray,100,255,cv2.THRESH_BINARY_INV)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

    if len(contours) != 0:

        # find the biggest countour (c) by the area
        c = max(contours, key = cv2.contourArea)
        epsilon = 0.1 * cv2.arcLength(c, True)
        corners = cv2.approxPolyDP(c, epsilon, True)
        # If our approximated contour has four points
        # draw the biggest contour (c) in green


    def order_points(pts):
        '''Rearrange coordinates to order:
          top-left, top-right, bottom-right, bottom-left'''
        rect = np.zeros((4, 2), dtype='float32')
        pts = np.array(pts)
        s = pts.sum(axis=1)
        # Top-left point will have the smallest sum.
        rect[0] = pts[np.argmin(s)]
        # Bottom-right point will have the largest sum.
        rect[2] = pts[np.argmax(s)]

        diff = np.diff(pts, axis=1)
        # Top-right point will have the smallest difference.
        rect[3] = pts[np.argmin(diff)]
        # Bottom-left will have the largest difference.
        rect[1] = pts[np.argmax(diff)]
        # return the ordered coordinates
        return rect.astype('int').tolist()


    pt_A,pt_B,pt_C,pt_D=order_points([corners[0][0],corners[1][0],corners[2][0],corners[3][0]])

    # Here, I have used L2 norm. You can use L1 also.
    width_AD = np.sqrt(((pt_A[0] - pt_D[0]) ** 2) + ((pt_A[1] - pt_D[1]) ** 2))
    width_BC = np.sqrt(((pt_B[0] - pt_C[0]) ** 2) + ((pt_B[1] - pt_C[1]) ** 2))
    maxWidth = max(int(width_AD), int(width_BC))


    height_AB = np.sqrt(((pt_A[0] - pt_B[0]) ** 2) + ((pt_A[1] - pt_B[1]) ** 2))
    height_CD = np.sqrt(((pt_C[0] - pt_D[0]) ** 2) + ((pt_C[1] - pt_D[1]) ** 2))
    maxHeight = max(int(height_AB), int(height_CD))

    input_pts = np.float32([pt_A, pt_B, pt_C, pt_D])
    output_pts = np.float32([[0, 0],
                            [0, maxHeight - 1],
                            [maxWidth - 1, maxHeight - 1],
                            [maxWidth - 1, 0]])

    # Getting the homography.
    M = cv2.getPerspectiveTransform(input_pts,output_pts)

    return cv2.warpPerspective(img,M,(maxWidth, maxHeight),flags=cv2.INTER_LINEAR)

In [ ]:
def remove_small_contours(img,small_area=1000):
  """this founction remove small countours from the image. used to remove the numbers

  Args:
    img (cv2 image format)

  Returns:
    img (cv2 image format)
  """

  mask = np.zeros(img.shape[:2], dtype=img.dtype)
  cnts, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  for c in cnts:
      area = cv2.contourArea(c)
      if area > small_area:
        x, y, w, h = cv2.boundingRect(c)
        cv2.drawContours(mask, [c], 0, (255), -1)

  return cv2.bitwise_and(img,img, mask= mask)


def get_cells(board_img):
    """corpes the cells from the board

    Args:
      board_img (cv2 image format) - board

    Returns:
      board_img (cv2 image format) - the board corped for the image
    """

    filter=cv2.imread("filter4.jpg")
    print(filter.shape)
    kernel=filter/np.sum(filter)

    gray = cv2.cvtColor(board, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,57,5)
    cv2_imshow(thresh)

    dst_ver =cv2.filter2D(thresh,-1,cv2.transpose(kernel))
    dst_hor =cv2.filter2D(thresh,-1,kernel)

    _,thresh_ver = cv2.threshold(dst_ver,30,255,cv2.THRESH_BINARY)
    _,thresh_hor = cv2.threshold(dst_hor,30,255,cv2.THRESH_BINARY)


    ver_lines=remove_small_contours(thresh_ver)
    hor_lines=remove_small_contours(thresh_hor)

    board_lines=np.maximum(ver_lines,hor_lines)
    cv2_imshow(thresh_hor)
    print('')
    cv2_imshow(hor_lines)
    cv2_imshow(board_lines)

    invert = 255 - board_lines
    cv2_imshow(invert)
    cnts,_ = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")

    newBoard = board.copy()
    rectangles=[]
    for c in cnts:
      if(cv2.contourArea(c)<0**2): continue
      # print(cv2.boundingRect(c))
      rectangles.append(cv2.boundingRect(c))
      x,y,w,h = cv2.boundingRect(c)
    print(len(rectangles))
    # for rectangle in rectangles:
    #   cv2_imshow(board[rectangle[0]:(rectangle[0] + rectangle[2]),rectangle[1]:(rectangle[1]+ rectangle[3])])

    a,b,_=board_img.shape

    rectangles = sorted(rectangles, key=lambda x: int(x[0]/a*9)/100+int(x[1]/b*9), reverse=False)
    i=0
    for c in rectangles:
      x,y,w,h=c
      cv2.rectangle(newBoard, (x, y), (x + w, y + h), (255,0,0), 1)
      cv2.putText(newBoard, str(i), (x,y+h),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA)
      i+=1

    cv2_imshow(newBoard)

    return rectangles




# sudoku functions

In [11]:
puzzle = Sudoku(3).difficulty(0.5)
def load_sudoku(board):
  return Sudoku(3, 3, board=board)
def solve_sudoku(sudoku):
  return puzzle.solve()

In [12]:
game = load_sudoku(puzzle.board)
print(game.board)
print(solve_sudoku(game).board)

[[None, 7, 9, 2, None, 4, None, None, None], [None, 1, 8, 3, None, 6, None, None, 2], [2, None, None, None, 8, None, 5, 6, None], [None, None, None, 9, 2, 7, None, 4, 6], [7, 3, None, 4, None, None, None, 2, 8], [4, None, None, 6, 3, None, None, 5, None], [3, 2, 4, 8, None, 1, None, None, 5], [None, None, 7, None, None, 2, 1, None, 9], [None, None, None, None, 6, 3, None, 8, 4]]
[[6, 7, 9, 2, 5, 4, 8, 1, 3], [5, 1, 8, 3, 7, 6, 4, 9, 2], [2, 4, 3, 1, 8, 9, 5, 6, 7], [1, 8, 5, 9, 2, 7, 3, 4, 6], [7, 3, 6, 4, 1, 5, 9, 2, 8], [4, 9, 2, 6, 3, 8, 7, 5, 1], [3, 2, 4, 8, 9, 1, 6, 7, 5], [8, 6, 7, 5, 4, 2, 1, 3, 9], [9, 5, 1, 7, 6, 3, 2, 8, 4]]
